In [ ]:
pip install --upgrade azureml-sdk

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

boston = load_boston()

X = boston.data[:,5].reshape(-1,1)
y = boston.target.reshape(-1,1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 0)


from sklearn.linear_model import LinearRegression

# Create a Linear regressor
model = LinearRegression()

# Train the model using the training sets
model.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

LinearRegression()

In [ ]:
import joblib

joblib.dump(model, 'housepricepredictor.joblib')

['housepricepredictor.joblib']

In [ ]:

from azureml.core import Workspace

NAME = "aml-test-ws" # The name of the workspace to be created
RESOURCE_GROUP = "aml-test" # The name of the resource group that will contain the workspace
LOCATION = "westeurope" # The data center location to deploy to
SUBSCRIPTION_ID = <subscription_id> # GUID identifying the subscription with which to deploy

ws = Workspace.create(
    name=NAME,
    resource_group=RESOURCE_GROUP,
    subscription_id=SUBSCRIPTION_ID,
    location=LOCATION,
    sku='basic', # Azure ML version (available: basic or enterprise)
    create_resource_group=True, # A new workspace will be created if it doesn't exist yet
    exist_ok=True, # The method succeeds if the workspace already exists
    show_output=True)

ws.write_config()

Performing interactive authentication. Please follow the instructions on the terminal.


Interactive authentication successfully completed.
Deploying StorageAccount with name amltestwstorage3b87a7355.
Deploying AppInsights with name amltestwinsightsc1e297d7.
Deployed AppInsights with name amltestwinsightsc1e297d7. Took 6.84 seconds.
Deploying KeyVault with name amltestwkeyvaulta35564c2.
Deployed KeyVault with name amltestwkeyvaulta35564c2. Took 26.67 seconds.
Deployed StorageAccount with name amltestwstorage3b87a7355. Took 26.58 seconds.
Deploying Workspace with name aml-test-ws.
Deployed Workspace with name aml-test-ws. Took 39.2 seconds.


In [ ]:
from azureml.core import Workspace
from azureml.core import Model


ws = Workspace.from_config("config.json")

model = Model.register(workspace = ws,
                       model_path ="housepricepredictor.joblib",
                       model_name = "house-price-predictor-model",
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.22.2',
                       description = "A simple lr model")

Registering model house-price-predictor-model


In [ ]:
%%writefile score.py

import joblib
import os
import json
import numpy as np

from azureml.core import Run
from azureml.core.model import Model


# The init() method is called once, when the web service starts up.
def init():
    """
    Initialize the scoring script
    """
    global model, run

    # Locate the model in Azure Machine Learning
    model_artifact = "housepricepredictor.joblib"
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_artifact)
    # AZUREML_MODEL_DIR is an environment variable created during deployment,
    # indicating the path to the folder containing the registerd moedels
    # (./azureml-models/$MODEL_NAME/$VERSION)

    # Load the model
    model = joblib.load(model_path)


# The run() method is called each time a request is made to the scoring API.
def run(data):
    """
    Make predictions
    """

    # Extract the input data from the request
    # (we expect it to be a 2D array, where every row
    # is a different sample for the model to process)
    input_data = json.loads(data)['data']

    # Run the model on the input data
    output = model.predict(np.array(input_data))

    # Prepare the results as a dictionary,
    # so that it can be JSON-serialized
    # (as the returned data needs to be JSON-serializable)
    result = {"price" : output.tolist()}

    return result

Overwriting score.py


In [ ]:

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment = Environment("my-custom-environment")

environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults>= 1.0.45', # mandatory dependency, contains the functionality needed to host the model as a web service
    'inference-schema[numpy-support]', # dependency for automatic schema generation (for parsing and validating input data)
    'joblib',
    'numpy',
    'scikit-learn'
])


In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=environment)

In [ ]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model

model_name = "house-price-predictor-model"
endpoint_name = "house-price-predictor-ep"

ws = Workspace.from_config()

model = Model(ws, name=model_name)

service = Model.deploy(workspace=ws,
                       name=endpoint_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  from ipykernel import kernelapp as app


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-10-09 05:45:47+00:00 Creating Container Registry if not exists..
2022-10-09 05:55:48+00:00 Registering the environment.
2022-10-09 05:55:51+00:00 Building image..
2022-10-09 06:06:09+00:00 Generating deployment configuration..
2022-10-09 06:06:11+00:00 Submitting deployment to compute..
2022-10-09 06:06:19+00:00 Checking the status of deployment house-price-predictor-ep..
2022-10-09 06:08:25+00:00 Checking the status of deployment house-price-predictor-ep..
2022-10-09 06:09:38+00:00 Checking the status of inference endpoint house-price-predictor-ep.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
import json
import requests

input_array = [[ 6.575 ]]
scoring_uri = <scoring_uri>

# Add the 'data' field
data = { "data" : input_array,
        "method" : "predict"} # Write it in the required format for the REST API

input_data = json.dumps(data) # Convert to JSON string

# Set the content type to JSON
headers = {"Content-Type": "application/json"}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

# Return the model output
result = json.loads(resp.text)
print(result)

{'price': [[25.051648512173124]]}


In [ ]:
print("Input data from dataframe: " + str(X[0]))
print("Output from dataframe: " + str(y[0]))

Input data from dataframe: [6.575]
Output from dataframe: [24.]
